connTask is a package used for the prediction of task evoked patterns of activity (derived from tFMRI data) using functional connectivity data

it is based on the work by Tavor et al (2016) https://science.sciencemag.org/content/352/6282/216

install: "pip install conntask_ni"

In [1]:
# pip install conntask_ni

In [9]:
import numpy as np
from sklearn.model_selection import GroupKFold
X = [('a','b'),(3,4),(5,6),(7,8),(9,10)]
y = None
groups = np.arange(len(X))
group_kfold = GroupKFold(n_splits=2)
for fold, (train_indices, test_indices) in enumerate(group_kfold.split(X, y, groups)):
    print(fold)
    print(train_indices)
    print(test_indices)
    print(np.array(X)[train_indices])

0
[1 3]
[0 2 4]
[['3' '4']
 ['7' '8']]
1
[0 2 4]
[1 3]
[['a' 'b']
 ['5' '6']
 ['9' '10']]


In [2]:
from src.conntask_ni import utils, extract_features, model_and_predict
import nibabel as nb
import numpy as np
import seaborn as sns
import os
import torchio as tio
import nilearn
import nilearn.image 
import nilearn.masking

In [35]:
sample = np.arange(2400).reshape(300,4,2)

In [52]:
def normalise_like_matlab_a(x):
    # normalising like matlab. used for consistency with s.jbabdi's matlab code
    dim = 0
    dims = x.shape # (voxels, subj, num_features)
    #print(dims) # (300, 4, 2)
    dimsize = dims[dim] #voxels
    dimrep = np.ones(len(dims), dtype=int) # (voxels, subj, num_features)
    #print(dimrep) # [1 1 1]
    dimrep[dim] = dimsize
    #print(dimrep) # [300   1   1]
    #print(x.shape) # (300, 4, 2)
    x = x - np.tile(x.mean(axis=0), reps=dimrep)
    #print(x.shape)
    x = x/np.tile(x.std(axis=0, ddof=1), reps=dimrep)
    #print(x)
    x[np.isnan(x)] = 0
    x[np.isinf(x)] = 0
    x = x/np.sqrt(dimsize - 1)
    #print(x)
    return x


In [54]:
a = normalise_like_matlab_a(sample)
a

array([[[-0.09966722, -0.09966722],
        [-0.09966722, -0.09966722],
        [-0.09966722, -0.09966722],
        [-0.09966722, -0.09966722]],

       [[-0.09900055, -0.09900055],
        [-0.09900055, -0.09900055],
        [-0.09900055, -0.09900055],
        [-0.09900055, -0.09900055]],

       [[-0.09833388, -0.09833388],
        [-0.09833388, -0.09833388],
        [-0.09833388, -0.09833388],
        [-0.09833388, -0.09833388]],

       ...,

       [[ 0.09833388,  0.09833388],
        [ 0.09833388,  0.09833388],
        [ 0.09833388,  0.09833388],
        [ 0.09833388,  0.09833388]],

       [[ 0.09900055,  0.09900055],
        [ 0.09900055,  0.09900055],
        [ 0.09900055,  0.09900055],
        [ 0.09900055,  0.09900055]],

       [[ 0.09966722,  0.09966722],
        [ 0.09966722,  0.09966722],
        [ 0.09966722,  0.09966722],
        [ 0.09966722,  0.09966722]]])

In [63]:
from sklearn.preprocessing import normalize

def normalise_like_matlab_b(x):
    # normalising like matlab. used for consistency with s.jbabdi's matlab code
    dim = 0
    dims = x.shape # (voxels, subj, num_features)
    dimsize = dims[dim] #voxels
    x = (x - x.mean(axis=0)) /x.std(axis=0, ddof=1)
    x[np.isnan(x)] = 0
    x[np.isinf(x)] = 0   
    x = x/np.sqrt(dimsize - 1)
    return x
psutil.Process().memory_info().rss / (1024 * 1024)

451.96484375

In [67]:
type([1,2,3]) == list

True

In [64]:
b= normalise_like_matlab_b(sample)

In [65]:
np.array_equal(a,b)

True

In [70]:
list(range(0,9323,10))

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 510,
 520,
 530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 810,
 820,
 830,
 840,
 850,
 860,
 870,
 880,
 890,
 900,
 910,
 920,
 930,
 940,
 950,
 960,
 970,
 980,
 990,
 1000,
 1010,
 1020,
 1030,
 1040,
 1050,
 1060,
 1070,
 1080,
 1090,
 1100,
 1110,
 1120,
 1130,
 1140,
 1150,
 1160,
 1170,
 1180,
 1190,
 1200,
 1210,
 1220,
 1230,
 1240,
 1250,
 1260,
 1270,
 1280,
 1290,
 1300,
 1310,
 1320,
 1330,
 1340,
 1350,
 1360,
 1370,
 1380,
 1390,
 1400,
 1410,
 1420,
 1430,
 1440,
 1450,
 1460,
 1470,
 1480,
 1490,
 1500,
 1510,
 1520,
 1530,
 1540,
 1550,
 1560,
 1570,
 1580

the first step is to extract functional connectivity features from the raw data (usually rsfMRI, couls be also naturalistic-stimuli fMRI or even tFMRI time-series data)

to do so, we use a set of spatially individual components, derived from a group ica procedure, to define define a set of networks that will be the basis of our feature set.

in this example, we use a set of 43 components dervied from the data of 100 HCP participants

### Group ICA map given by the author (HCP)

In [3]:
cifti_orig = nb.load('src/conntask_ni/files/ica_both_hemis_45_comp_cleaned.dtseries.nii') # i keep t
ica = np.asarray(cifti_orig.get_fdata())

In [4]:
ica.shape # should be number of components * voxels

(43, 91282)

now we are trying to make group ica file of UKB as this

## Filter out bad components in GroupICA map

### component 25 (->21)

In [8]:
nifti_orig = nb.load('/global/cfs/cdirs/m4244/junbeom/UKBiobank_BrainImaging_GroupMeanTemplates/rfMRI_ICA_d25.nii.gz')

file = open('/global/cfs/cdirs/m4244/junbeom/UKBiobank_BrainImaging_GroupMeanTemplates/rfMRI_GoodComponents_d25_v1.txt','r')
goodcomponent25 = [ int(i)-1 for i in file.readline().strip().split()]
print(goodcomponent25)
file.close()

ica_good_21 = nilearn.image.index_img(nifti_orig, goodcomponent25)
ica_good_21.shape

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


(91, 109, 91, 21)

### component 100 (-> 55)

In [9]:
nifti_orig = nb.load('/global/cfs/cdirs/m4244/junbeom/UKBiobank_BrainImaging_GroupMeanTemplates/rfMRI_ICA_d100.nii.gz')
ica100 = np.asarray(nifti_orig.get_fdata())

file = open('/global/cfs/cdirs/m4244/junbeom/UKBiobank_BrainImaging_GroupMeanTemplates/rfMRI_GoodComponents_d100_v1.txt','r')
goodcomponent100 = [ int(i)-1 for i in file.readline().strip().split()] # -1 because this file starts from 1, but index starts from 0
print(goodcomponent100)
file.close()

ica_good_55 = nilearn.image.index_img(nifti_orig, goodcomponent100) # filter out 45 bad group ica components
ica_good_55.shape

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 47, 48, 49, 51, 52, 56, 57, 59, 62, 63, 92]


(91, 109, 91, 55)

next, we go over all participants in our data set and perform the 2 steps of the feature extraction procedure:

1) dual regression of group ica components, to yield participant-specific representation of the group-ica

2) a weighted seed2voxel procedure on the the components from the previous step

## Masking group ICA map

load ICBM 152 nonlin symmetric 6th generation

In [13]:
mask_file_nifti = nb.load('/global/cfs/cdirs/m4244/registration/icbm_avg_152_t1_tal_nlin_symmetric_VI_mask.nii')
mask_file=mask_file_nifti.get_fdata().astype(int)
mask_file.shape

(193, 229, 193)

In [14]:
target_affine = nifti_orig.affine
target_shape = nifti_orig.shape[:3]
MNI152_mask = nilearn.image.resample_img(mask_file_nifti,target_affine=target_affine,target_shape=target_shape,interpolation='nearest')


In [15]:
MNI152_mask.shape, MNI152_mask.affine

((91, 109, 91),
 array([[  -2.,    0.,    0.,   90.],
        [   0.,    2.,    0., -126.],
        [   0.,    0.,    2.,  -72.],
        [   0.,    0.,    0.,    1.]]))

In [7]:
nb.save(MNI152_mask,'MNI_152_mask.nii.gz')

NameError: name 'MNI152_mask' is not defined

preprocessed MNI152 mask

In [12]:
import nibabel as nb
MNI152_mask = nb.load('output/UKB/mask/MNI_152_mask.nii.gz')

In [20]:
target_affine = MNI152_mask.affine
target_shape = MNI152_mask.shape[:3]

ICBM152_graymatter = nilearn.image.resample_img(nilearn.datasets.fetch_icbm152_brain_gm_mask(),target_affine=target_affine,target_shape=target_shape,interpolation='nearest')
nb.save(ICBM152_graymatter,'output/UKB/mask/ICBM_152_GM_mask.nii.gz')

In [38]:
wm_raw = nilearn.datasets.load_mni152_wm_mask()
wm_raw.get_fdata().astype(int)

{0, 1}

In [33]:
set(.get_fdata().flatten())

{0.0, 1.0}

In [26]:
set(ICBM152_graymatter.get_fdata().flatten())

{0.0, 1.0}

In [34]:
#set(nb.load(nilearn.datasets.fetch_icbm152_2009()['wm']).get_fdata().flatten())

In [27]:
target_affine = MNI152_mask.affine
target_shape = MNI152_mask.shape[:3]

ICBM152_wm = nilearn.image.resample_img(nilearn.datasets.fetch_icbm152_2009()['wm'],target_affine=target_affine,target_shape=target_shape,interpolation='nearest')
#nb.save(ICBM152_wm,'output/UKB/mask/MNI_152_WM_mask.nii.gz')

AttributeError: module 'nilearn.datasets' has no attribute 'fetch_icbm152_brain_wm_mask'

Component 21

In [53]:
# # add dimension to add channel
# mask_file = mask_file[np.newaxis,:,:,:]

# # resize from (193,229,193) to (91,109,91)
# resize_tio = tio.transforms.Resize((91,109,91),image_interpolation='nearest')
# MNI152_mask_tio = resize_tio(mask_file)

# # remove channel
# MNI152_mask_tio = np.squeeze(MNI152_mask_tio)


In [12]:
ica_good_21.shape

(91, 109, 91, 21)

In [11]:
masked_pixels = nilearn.masking.apply_mask(ica_good_21, mask_img=MNI152_mask)
masked_pixels.shape

(21, 237969)

In [18]:
np.save('UKB_groupica_21_comp_masked.npy',masked_pixels)

Component 55

In [10]:
ica_good_55.shape

(91, 109, 91, 55)

In [17]:
masked_pixels = nilearn.masking.apply_mask(ica_good_55, mask_img=MNI152_mask)
masked_pixels.shape

(55, 237969)

In [15]:
np.save('UKB_groupica_55_comp_masked.npy',masked_pixels)

load sample rsfMRI in MNI space

In [21]:
sample = nb.load('/global/cfs/cdirs/m4244/registration/20227_1_MNI/1000246_20227_2_0_rsfMRI_MNI_space.nii.gz')

In [22]:
sample.shape, sample.affine

((91, 109, 91, 490),
 array([[  -2.,    0.,    0.,   90.],
        [   0.,    2.,    0., -126.],
        [   0.,    0.,    2.,  -72.],
        [   0.,    0.,    0.,    1.]]))

In [23]:
masked_pixels = nilearn.masking.apply_mask(sample, mask_img=MNI152_mask)
masked_pixels.shape

(490, 237969)

In [24]:
# masked_pixels_tio = sample.get_fdata()[MNI152_mask_tio == 1].swapaxes(0,1)
# masked_pixels_tio.shape

### check whether mask file and registered rs-fMRI image matches

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

def explore_3d_image(x, y, z):
    #plt.figure(figsize=(8,10))
    # plt.title('Explore Layers of MRI', fontsize=18)
    
    fig, axes = plt.subplots(1, 3, figsize=(8, 12))
    
    axes[0].imshow(MNI152_mask.get_fdata()[x, :, :], cmap='gist_heat')
    axes[0].imshow(sample.get_fdata()[ x, :, :,0], cmap='gray',alpha=0.4)
    axes[1].imshow(MNI152_mask.get_fdata()[:, y, :], cmap='gist_heat')
    axes[1].imshow(sample.get_fdata()[:, y, :,0], cmap='gray',alpha=0.4)
    axes[2].imshow(MNI152_mask.get_fdata()[:, :, z], cmap='gist_heat')
    axes[2].imshow(sample.get_fdata()[ :, :, z,0], cmap='gray',alpha=0.4)
    
    #plt.axis('off')
    plt.tight_layout()
    plt.show()
    #return layer
 
# Run the ipywidgets interact() function to explore the data
interact(explore_3d_image, x=(0, MNI152_mask.shape[0]-1),y=(0, MNI152_mask.shape[1]-1),z=(0, MNI152_mask.shape[2]-1))

interactive(children=(IntSlider(value=45, description='x', max=90), IntSlider(value=54, description='y', max=1…

<function __main__.explore_3d_image(x, y, z)>

torchio (not used)

In [54]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# from PIL import Image

# import os
# import json
# import numpy as np
# from matplotlib.colors import LinearSegmentedColormap

# import torchvision
# from torchvision import models
# from torchvision import transforms

# from captum.attr import IntegratedGradients
# from captum.attr import GradientShap
# from captum.attr import Occlusion
# from captum.attr import NoiseTunnel
# from captum.attr import visualization as viz
# import matplotlib.pyplot as plt
# from matplotlib.colors import LogNorm
# import itk              # 2D 및 3D 이미지 시각화 라이브러리
# import itkwidgets
# from ipywidgets import interact, interactive, IntSlider, ToggleButtons

# def explore_3d_image(x, y, z):
#     #plt.figure(figsize=(8,10))
#     # plt.title('Explore Layers of MRI', fontsize=18)
    
#     fig, axes = plt.subplots(1, 3, figsize=(8, 12))
    
#     axes[0].imshow(MNI152_mask_tio[x, :, :], cmap='gist_heat')
#     axes[0].imshow(sample.get_fdata()[ x, :, :,0], cmap='gray',alpha=0.4)
#     axes[1].imshow(MNI152_mask_tio[:, y, :], cmap='gist_heat')
#     axes[1].imshow(sample.get_fdata()[:, y, :,0], cmap='gray',alpha=0.4)
#     axes[2].imshow(MNI152_mask_tio[:, :, z], cmap='gist_heat')
#     axes[2].imshow(sample.get_fdata()[ :, :, z,0], cmap='gray',alpha=0.4)
    
#     #plt.axis('off')
#     plt.tight_layout()
#     plt.show()
#     #return layer
 
# # Run the ipywidgets interact() function to explore the data
# interact(explore_3d_image, x=(0, MNI152_mask.shape[0]-1),y=(0, MNI152_mask.shape[1]-1),z=(0, MNI152_mask.shape[2]-1))

#should not use this method

interactive(children=(IntSlider(value=45, description='x', max=90), IntSlider(value=54, description='y', max=1…

<function __main__.explore_3d_image(x, y, z)>

# check whether mask file and AAL Templete image matches

In [71]:
import nibabel as nb

In [72]:
nb.load('src/conntask_ni/files/Schaefer2018_100Parcels_7Networks_order.dtseries.nii').shape

(1, 91282)

In [75]:
set(np.squeeze(nb.load('src/conntask_ni/files/Schaefer2018_100Parcels_7Networks_order.dtseries.nii').get_fdata()))

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0}

Schaefer2018_100Parcels_7Networks

In [7]:
import nilearn.masking
ROI_Sch_100P_7N = nb.load('Schaefer2018_ROIs/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz')

In [10]:
masked_ROI_Sch_100P_7N = nilearn.masking.apply_mask(ROI_Sch_100P_7N, mask_img=MNI152_mask)
np.save('masked_ROI_Sch_100P_7N.npy',masked_ROI_Sch_100P_7N[np.newaxis,:])

In [11]:
np.load('result/UKB/features-d21/1000246_20227_2_0_features_21_comps.npy').shape

(21, 237969)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

def explore_3d_image(x, y, z):
    #plt.figure(figsize=(8,10))
    # plt.title('Explore Layers of MRI', fontsize=18)
    
    fig, axes = plt.subplots(1, 3, figsize=(8, 12))
    
    axes[0].imshow(MNI152_mask.get_fdata()[x, :, :], cmap='gray',alpha=0.4)
    axes[0].imshow(ROI_Sch_100P_7N.get_fdata()[ x, :, :], cmap='gist_heat',alpha=0.7)
    axes[1].imshow(MNI152_mask.get_fdata()[:, y, :], cmap='gray',alpha=0.4)
    axes[1].imshow(ROI_Sch_100P_7N.get_fdata()[:, y, :], cmap='gist_heat',alpha=0.7)
    axes[2].imshow(MNI152_mask.get_fdata()[:, :, z], cmap='gray',alpha=0.4)
    axes[2].imshow(ROI_Sch_100P_7N.get_fdata()[ :, :, z], cmap='gist_heat',alpha=0.7)
    
    #plt.axis('off')
    plt.tight_layout()
    plt.show()
    #return layer
 
# Run the ipywidgets interact() function to explore the data
interact(explore_3d_image, x=(0, MNI152_mask.shape[0]-1),y=(0, MNI152_mask.shape[1]-1),z=(0, MNI152_mask.shape[2]-1))

interactive(children=(IntSlider(value=45, description='x', max=90), IntSlider(value=54, description='y', max=1…

<function __main__.explore_3d_image(x, y, z)>

AAL

In [3]:
ROI_AAL = nb.load('ROI_MNI_V7.nii')
ROI_AAL.shape

(91, 109, 91)

In [24]:
import nilearn.masking
masked_ROI_AAL = nilearn.masking.apply_mask(ROI_AAL, mask_img=MNI152_mask)
np.save('masked_AAL_ROI.npy',masked_ROI_AAL[np.newaxis,:])

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

def explore_3d_image(x, y, z):
    #plt.figure(figsize=(8,10))
    # plt.title('Explore Layers of MRI', fontsize=18)
    
    fig, axes = plt.subplots(1, 3, figsize=(8, 12))
    
    axes[0].imshow(MNI152_mask.get_fdata()[x, :, :], cmap='gray',alpha=0.4)
    axes[0].imshow(ROI_AAL.get_fdata()[ x, :, :], cmap='gist_heat',alpha=0.7)
    axes[1].imshow(MNI152_mask.get_fdata()[:, y, :], cmap='gray',alpha=0.4)
    axes[1].imshow(ROI_AAL.get_fdata()[:, y, :], cmap='gist_heat',alpha=0.7)
    axes[2].imshow(MNI152_mask.get_fdata()[:, :, z], cmap='gray',alpha=0.4)
    axes[2].imshow(ROI_AAL.get_fdata()[ :, :, z], cmap='gist_heat',alpha=0.7)
    
    #plt.axis('off')
    plt.tight_layout()
    plt.show()
    #return layer
 
# Run the ipywidgets interact() function to explore the data
interact(explore_3d_image, x=(0, MNI152_mask.shape[0]-1),y=(0, MNI152_mask.shape[1]-1),z=(0, MNI152_mask.shape[2]-1))

interactive(children=(IntSlider(value=45, description='x', max=90), IntSlider(value=54, description='y', max=1…

<function __main__.explore_3d_image(x, y, z)>

In [33]:
tmap = nb.load('/global/cfs/cdirs/m4244/registration/20249_unzip_1_beta_MNI/1000254_20249_2_0/zstat1_MNI_space.nii.gz')

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

def explore_3d_image(x, y, z):
    #plt.figure(figsize=(8,10))
    # plt.title('Explore Layers of MRI', fontsize=18)
    
    fig, axes = plt.subplots(1, 3, figsize=(8, 12))
    
    axes[0].imshow(MNI152_mask.get_fdata()[x, :, :], cmap='gist_heat')
    axes[0].imshow(tmap.get_fdata()[ x, :, :], cmap='gray',alpha=0.4)
    axes[1].imshow(MNI152_mask.get_fdata()[:, y, :], cmap='gist_heat')
    axes[1].imshow(tmap.get_fdata()[:, y, :], cmap='gray',alpha=0.4)
    axes[2].imshow(MNI152_mask.get_fdata()[:, :, z], cmap='gist_heat')
    axes[2].imshow(tmap.get_fdata()[ :, :, z], cmap='gray',alpha=0.4)
    
    #plt.axis('off')
    plt.tight_layout()
    plt.show()
    #return layer
 
# Run the ipywidgets interact() function to explore the data
interact(explore_3d_image, x=(0, MNI152_mask.shape[0]-1),y=(0, MNI152_mask.shape[1]-1),z=(0, MNI152_mask.shape[2]-1))

interactive(children=(IntSlider(value=45, description='x', max=90), IntSlider(value=54, description='y', max=1…

<function __main__.explore_3d_image(x, y, z)>